In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Installations
%%capture

!pip install decord
# !git clone https://github.com/eldhusfifl07/nmaDL-project.git
# %cd /content/nmaDL-project/resnetBasline

current directory set to

```
# !git clone https://github.com/eldhusfifl07/nmaDL-project.git
```
Change directory to Baseline Run for output


In [ ]:
#@title Imports
import glob
import numpy as np
import pandas as pd
import urllib
import torch
import cv2
import os
import argparse
import time
import random
import matplotlib.pyplot as plt
import nibabel as nib
import pickle
import sys 
sys.path.append("../")
from tqdm import tqdm
# from utils import *

from torchvision import transforms as trn
import torchvision.models as models
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
import torch.nn.functional as F
from torch.autograd import Variable as V
from torch.utils.data import Dataset, DataLoader

import albumentations as A
# from albumentations.pytorch import ToTensorV2

from PIL import Image
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, IncrementalPCA
from scipy.stats import ortho_group

from nilearn import datasets
from nilearn import surface
from nilearn import plotting

from decord import VideoReader
from decord import cpu






/usr/local/lib/python3.7/dist-packages/nilearn/datasets/__init__.py:89: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


Get the dropbox link to download dataset by filling this [google form](https://forms.gle/qq9uqqu6SwN8ytxQ9). 

In [ ]:
#@title PCA code



seed = 42
# Torch RNG
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
# Python RNG
np.random.seed(seed)
random.seed(seed)
def do_PCA_and_save(activations_dir, save_dir, layer):
  """This function preprocesses Neural Network features using PCA and save the results
  in  a specified directory.
  Parameters
  ----------
  activations_dir : str
      save path for extracted features.
  save_dir : str
      save path for extracted PCA features.
  Returns: x_train 
  """

  layers = [layer]
  n_components = 0.95

  if not os.path.exists(save_dir):
      os.makedirs(save_dir)

  for layer in tqdm(layers):
      activations_file_list = glob.glob(activations_dir +'/*'+layer+'.npy')
      activations_file_list.sort()
      feature_dim = np.load(activations_file_list[0])
      # feature_dim = feature_dim.flatten()
      x = np.zeros((len(activations_file_list),feature_dim.shape[0]))

      for i,activation_file in enumerate(activations_file_list):
          temp = np.load(activation_file)
          # temp = temp.flatten()
          x[i,:,:,:] = temp
      x_train = x[:1000,:,:,:]
      x_test = x[1000:,:,:,:]

      start_time = time.time()
      x_test = StandardScaler().fit_transform(x_test)
      x_train = StandardScaler().fit_transform(x_train)
      ipca = PCA(n_components=n_components,random_state=seed)
      ipca.fit(x_train)

      x_train = ipca.transform(x_train)
      x_test = ipca.transform(x_test)
      train_save_path = os.path.join(save_dir,"train_"+layer)
      test_save_path = os.path.join(save_dir,"test_"+layer)
      np.save(train_save_path,x_train)
      np.save(test_save_path,x_test)
  return ipca.n_components_
  

def plot_principal_component(pca, i):
  # Get PC representation as a subspace with size proportional to the corresponding singular value
  size = np.sqrt(pca.singular_values_[i])
  pc = np.outer(pca.components_[i], np.array([-1,1])) * size
  color_scheme = ["rgb(189,6,96)", "rgb(6,189,99)", "rgb(6,96,189)"]

  return go.Scatter3d(x=pc[0], y=pc[1], z=pc[2], mode="lines", opacity=.5,
                        line=dict(color=color_scheme[i], width=2*size), name='PC {}'.format(i+1))

In [ ]:
import os, os.path
!cd '/content/'
DIR = './layer_7'
print(len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

1102


In [ ]:
#@title get PCA and save activations

save_dir = "/content/resnetPCAs/"

activations_dir = "/content/layer_9"
if not os.path.exists(activations_dir):
    os.makedirs(activations_dir)

pca_dir = os.path.join(save_dir, 'pca_new')
print("-------------performing  PCA----------------------------")
do_PCA_and_save(activations_dir, pca_dir, "layer_9")

-------------performing  PCA----------------------------


  0%|          | 0/1 [00:00<?, ?it/s]


IndexError: ignored

# Unnecessary or test **codes**

In [ ]:
!unzip '/content/drive/MyDrive/Colab Notebooks/DumboOctopi/AlgonautsDataExtraction/layer_3.zip'

Archive:  /content/drive/MyDrive/Colab Notebooks/DumboOctopi/AlgonautsDataExtraction/layer_3.zip
  inflating: layer_3/0001_0-0-1-6-7-2-8-0-17500167280_layer_3.npy  
  inflating: layer_3/0002_0-0-4-3146384004_layer_3.npy  
  inflating: layer_3/0003_0-0-8-1-2-4-0-0-3500812400_layer_3.npy  
  inflating: layer_3/0004_0-1-2-9-1-9-9-5-2401291995_layer_3.npy  
  inflating: layer_3/0005_0-1-7-8-3-6-4-8-4401783648_layer_3.npy  
  inflating: layer_3/0006_0-1-9-9-8-6-5-9-4701998659_layer_3.npy  
  inflating: layer_3/0007_0-2-3-14056753023_layer_3.npy  
  inflating: layer_3/0008_0-2-9-6-5-7-2-3-16002965723_1_layer_3.npy  
  inflating: layer_3/0009_0-3-5-4-1-3-6-2-8603541362_layer_3.npy  
  inflating: layer_3/0010_0-3-6-10842732036_layer_3.npy  
  inflating: layer_3/0011_0-3-6-2-3-3-4-4-8103623344_layer_3.npy  
  inflating: layer_3/0012_0-5-3-7-6-2-8-4-24105376284_layer_3.npy  
  inflating: layer_3/0013_0-5-3-8-6-8-2-6-4805386826_layer_3.npy  
  inflating: layer_3/0014_0-5-6-8272016056_edit_layer_3

In [ ]:
!rm -rf './layer_4'

In [ ]:
dat = np.load('/content/resnetPCAs/pca_966/test_layer_5.npy')
dat.shape

In [ ]:
layers = ['layer_1','layer_2','layer_3','layer_4','layer_5','layer_6','layer_7','layer_8', 'layer_9', 'layer_10']
for layer in layers:
  x = np.load(f"/content/drive/MyDrive/Colab Notebooks/DumboOctopi/AlgonautsDataExtraction/resnetBasline/activations_resnet18/0001_0-0-1-6-7-2-8-0-17500167280_{layer}.npy")
  print(x.shape)

(1, 64, 112, 112)
(1, 64, 112, 112)
(1, 64, 112, 112)
(1, 64, 56, 56)
(1, 64, 56, 56)
(1, 128, 28, 28)
(1, 256, 14, 14)
(1, 512, 7, 7)
(1, 512, 1, 1)
(1, 1000)


In [ ]:
!ls '/content'

drive  layer_5	resnet18.pth  resnetPCAs  sample_data


In [ ]:
def do_PCA(activations_dir):
    """This function preprocesses Neural Network features using PCA and save the results
    in  a specified directory
.
    Parameters
    ----------
    activations_dir : str
        save path for extracted features.
    save_dir : str
        save path for extracted PCA features.
    """

    layers = ['layer_5']
    n_components = 100
    for layer in layers:
        activations_file_list = glob.glob(activations_dir +'/*'+layer+'.npy')
        activations_file_list.sort()
        feature_dim = np.load(activations_file_list[0])
        feature_dim = feature_dim.flatten()
        x = np.zeros((len(activations_file_list),feature_dim.shape[0]))
        for i,activation_file in enumerate(activations_file_list):
            temp = np.load(activation_file)
            temp = temp.flatten()
            x[i,:] = temp
        start_time = time.time()
        x = StandardScaler().fit_transform(x)
        # ipca = PCA(n_components=n_components,random_state=np.random.seed(1))
        # ipca.fit(x)

        # x = ipca.transform(x)

        return x

        
X = do_PCA('/content/layer_5')

In [ ]:
feature_dim = np.load('/content/layer_4/0001_0-0-1-6-7-2-8-0-17500167280_layer_4.npy')
print(feature_dim.shape)
feature_dim = feature_dim.flatten()
(feature_dim.shape)

(1, 64, 56, 56)


(200704,)

In [ ]:
from pandas import DataFrame
comp = [944, 946, 948]
for n in comp:
  pca = PCA(n_components=n).fit(X)
  print(sum(pca.explained_variance_ratio_))

0.949963334383559


In [ ]:
from pandas import DataFrame

pca = PCA(n_components=3).fit(X) # change the number of n_components to search for the best subspace

ev = pca.singular_values_**2
DataFrame(np.array([ev, ev/sum(ev), pca.explained_variance_ratio_]),
          columns=["PC 1", "PC 2", "PC3"],
          index=["Eigenvalues", "Explained Variance", "sklearn's Explained Variance"])
variance = list(np.around(100*pca.explained_variance_ratio_, 2)) + [100]

fig = make_subplots(rows=1, cols=2, subplot_titles=[r"$\text{Eigenvalues}$", r"$\text{Cumulative Explained Variance}$"],
                    specs=[[{'type': 'Bar'}, {'type': 'Waterfall'}]])

fig.add_traces([go.Bar(x=['PC1', 'PC2', 'PC3'], y=pca.singular_values_, marker_color = color_scheme),
                go.Waterfall(x=["PC1", "PC2", "PC3", "Total"],
                             y=variance,
                             text=[f"{v}%" for v in variance],
                             textposition = "outside",
                             totals = {"marker":{"color":"black"}},
                             measure = ["relative", "relative", "relative", "total"])],
               rows=[1,1], cols=[1,2])

fig.add_shape(type="rect", xref="x", yref="y", x0=-0.4, x1=0.4, y0=0.0, y1=fig.data[1].y[0], 
              fillcolor=color_scheme[0], line=dict(color=color_scheme[0]), opacity=1,row=1, col=2)
fig.add_shape(type="rect", xref="x", yref="y", x0=0.6, x1=1.4, y0=fig.data[1].y[0], y1=fig.data[1].y[0]+fig.data[1].y[1],
              fillcolor=color_scheme[1], line=dict(color=color_scheme[1]), opacity=1, row=1, col=2)
fig.add_shape(type="rect", xref="x", yref="y", x0=1.6, x1=2.4, y0=fig.data[1].y[0]+fig.data[1].y[1], y1=fig.data[1].y[0]+fig.data[1].y[1]+fig.data[1].y[2], 
              fillcolor=color_scheme[2], line=dict(color=color_scheme[2]), opacity=1, row=1, col=2)

fig.update_layout(showlegend=False, title=r"$\text{(4) PCA Explained Variance}$", margin=dict(t=100))
fig.show()